In [68]:
#EXAMPLE CODE FOR CHECKING IF WEBSITE LETS ME TO SCRAPE 

from bs4 import BeautifulSoup
import requests
import pandas

# Get data by pretending we are a browser like this.
#https://stackoverflow.com/questions/52068920/python-requests-beautifulsoup-bot-detection
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}

url = 'https://www.nbcnews.com/politics/supreme-court/supreme-court-rejects-rfk-jr-groups-attempt-protect-anti-covid-vaccine-rcna181061'  #URL FOR SPECIFIC PAGE - I changed url for each article

response = requests.get(url, headers=headers)
# response 200 means that it is working
print(response)

<Response [200]>


## 1) CNN 

https://www.cnn.com/2025/03/07/health/hhs-cdc-vaccines-autism/index.html

CSV FILE : realnews1_CNN.csv

In [71]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

#I am creaating a function for scraping article
def scrape_article(url):
#Added this part to run browser in the background
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options) 
    driver.get(url)
    time.sleep(3)


#BODY TEXT
    try:
        paragraphs = driver.find_elements(By.CSS_SELECTOR, 'div.article__content p')
        body_text = "\n".join([p.text for p in paragraphs])  #Added this part after getting error for paragraphs
    except:
        body_text = "none"

#IMAGE url and IMAGE ALT
    image_urls = []
    image_alts = []
    try:
        #Image loop
        images = driver.find_elements(By.TAG_NAME, 'img')
        for image in images:
            image_url = image.get_attribute('src')
            image_alt = image.get_attribute('alt')
            image_urls.append(image_url if image_url else "No image URL found")
            image_alts.append(image_alt if image_alt else "No image alt text found")
    except:
        image_alts = ["none"]
        image_urls = ["none"]

    driver.quit()

    return {
        "Text": body_text,
        "Image Alt Texts": image_alts,
        "Image URLs": image_urls
    }
#saving data to csv file
def save_to_csv(data, filename):
    df = pd.DataFrame([data])
    df.to_csv(filename, index=True, index_label="Index", encoding="utf-8-sig")

#calling functions
if __name__ == "__main__":
    url = "https://www.cnn.com/2025/03/07/health/hhs-cdc-vaccines-autism/index.html"
    article = scrape_article(url)
    save_to_csv(article, "realnews1_CNN.csv")
    print("done")

done


## 2) NewsNation -Youtube

https://www.youtube.com/watch?v=QBvmqwO1XdU

__I used APIFY, "YouTube Transcript Extractor:Video Text" actor to scrape text from the video. Good thing about this actor that you can pick the columns you want to keep in your csv file.__

CSV FILE:  realnews2_NewsNation_Youtube.csv

## 3) Yahoo News

https://www.yahoo.com/news/robert-f-kennedy-jr-falsely-090041824.html?fr=sycsrp_catchall 

CSV FILE: 'realnews3_Yahoo.csv'

In [74]:
from bs4 import BeautifulSoup
import requests
import pandas

# Get data by pretending we are a browser like this.
#https://stackoverflow.com/questions/52068920/python-requests-beautifulsoup-bot-detection
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
url = 'https://www.yahoo.com/news/robert-f-kennedy-jr-falsely-090041824.html?fr=sycsrp_catchall'
response = requests.get(url, headers=headers)
# response 200 means that it is working
#print(response)
if response.status_code == 200:
    # I am creating an empty dictionary here
    output = {"Title": [], "Author": [], "Text": []}

    soup = BeautifulSoup(response.content, 'lxml')
    #Capture the source the article from the image alt text
    #Scrape all images from the article
    #source = soup.findAll('img')
    #Print the alt text of the first image
    #print(source[1]['alt'])
    #output['Source'].append(source[1]['alt'])
    # Capture the title of the article
    title = soup.find('h1').get_text()
    #print(title)
    output['Title'].append(title)
    #Scrape the name of the article's author if data is there
    author = soup.find('div', class_='text-sm font-semibold').get_text()
    #print(author)
    output['Author'].append(author)
    #Scrape the article text
    article = soup.find('div', class_='col-body').get_text()
    #print(article)
    output['Text'].append(article)

#Save results to a csv file
results = pandas.DataFrame(output)
results.to_csv('realnews3_Yahoo.csv', index=True, index_label="Index",encoding="utf-8-sig")

print("done")



done


## FORBES 

https://www.forbes.com/sites/ariannajohnson/2023/05/22/new-studies-find-covid-vaccines-are-safe-for-kids-here-are-the-different-types-of-vaccines-and-how-they-work/?ctpv=searchpage

CSV FILE: realnews4_Forbes.csv

In [36]:
#I am creaating a function for scraping article
def scrape_article(url):
    # headless browser
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)

    driver.get(url)
    time.sleep(3)

 
    # BODY TEXT
    try:
        paragraphs = driver.find_elements(By.CSS_SELECTOR, 'div.article-body p')
        body_text = "\n".join([p.text for p in paragraphs])
    except:
        body_text = "none"

    # IMAGE ALT + URL
    try:
        image = driver.find_element(By.TAG_NAME, 'img')
        image_alt = image.get_attribute('alt')
        image_url = image.get_attribute('src')
    except:
        image_alt = "none"
        image_url = "none"

    driver.quit()

    return {
        "Text": body_text,
        "Image Alt Text": image_alt,
        "Image URL": image_url
    }

def save_to_csv(data, filename):
    df = pd.DataFrame([data])
    df.to_csv(filename, index=True, index_label="Index", encoding="utf-8-sig")

if __name__ == "__main__":
    url = "https://www.forbes.com/sites/ariannajohnson/2023/05/22/new-studies-find-covid-vaccines-are-safe-for-kids-here-are-the-different-types-of-vaccines-and-how-they-work/?ctpv=searchpage"
    article = scrape_article(url)
    save_to_csv(article, 'realnews4_Forbes.csv')
    print("done")


done


## 5) CBS News - Youtube Video

https://www.youtube.com/watch?v=ZIvWuB1ucQ8

__I used APIFY, "YouTube Transcript Extractor:Video Text" actor to scrape text from the video. Good thing about this actor that you can pick the columns you want to keep in your csv file.__

CSV FILE: realnews5_Youtube_CBS.csv

## 6) WCVB Channel 5 Boston - Youtube Video

https://www.youtube.com/watch?v=ofQYWI28ZAQ

__I used APIFY, "YouTube Transcript Extractor:Video Text" actor to scrape text from the video. Good thing about this actor that you can pick the columns you want to keep in your csv file.__ 
CSV FILE : realnews6_Youtube_WCVB.csv

## COMBINING FILES

In [78]:
import pandas as pd

#Load each CSV
df1 = pd.read_csv('realnews1_CNN.csv')
df2 = pd.read_csv('realnews2_NewsNation_Youtube.csv')
df3 = pd.read_csv('realnews3_Yahoo.csv')
df4 = pd.read_csv('realnews4_Forbes.csv')
df5 = pd.read_csv('realnews5_Youtube_CBS.csv')
df6 = pd.read_csv('realnews6_Youtube_WCVB.csv')

# Combine them
combined_df = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)

# Save to a new file
combined_df.to_csv('Combined_RealNews.csv', index=False, encoding='utf-8-sig')

print("done")

done
